# 01: Quickstart — From Zero to Enforced Architecture

You have a TypeScript project. It compiles fine, but nothing stops a developer from importing `fs` in the domain layer or calling the database from a controller.

This notebook walks through adding KindScript enforcement:

**Part 1 — Existing project:** `ksc infer` detects your architecture and generates definitions

**Part 2 — New project:** Write definitions first, then `ksc scaffold` creates the structure

**Part 3 — Advanced:** Standalone members, nested structure, path overrides

**Prerequisites:** Run `npx tsc` from the project root to compile the CLI.

## Setup

In [ ]:
const PROJECT_ROOT = Deno.cwd().replace(/\/notebooks$/, "");
const KSC = PROJECT_ROOT + "/dist/infrastructure/cli/main.js";

async function ksc(...args: string[]): Promise<{ code: number; output: string }> {
  const cmd = new Deno.Command("node", {
    args: [KSC, ...args],
    stdout: "piped",
    stderr: "piped",
  });
  const { code, stdout, stderr } = await cmd.output();
  const output = (new TextDecoder().decode(stdout) + new TextDecoder().decode(stderr)).trim();
  if (output) console.log(output);
  console.log(`\nExit code: ${code}`);
  return { code, output };
}

async function tree(dir: string): Promise<void> {
  const cmd = new Deno.Command("find", {
    args: [dir, "-type", "f"],
    stdout: "piped",
  });
  const { stdout } = await cmd.output();
  const files = new TextDecoder().decode(stdout).trim().split("\n")
    .map(f => f.replace(dir + "/", ""))
    .sort();
  console.log(files.join("\n"));
}

console.log("CLI path:", KSC);

---

## Part 1: Existing Project — Detect and Enforce

We'll create a small Clean Architecture project, then use KindScript to detect the pattern, generate definitions, and enforce contracts.

### Step 0: Create a realistic project

In [ ]:
const DEMO = Deno.makeTempDirSync({ prefix: "ksc-quickstart-" });
console.log("Demo project:", DEMO);

Deno.mkdirSync(`${DEMO}/src/domain`, { recursive: true });
Deno.mkdirSync(`${DEMO}/src/application`, { recursive: true });
Deno.mkdirSync(`${DEMO}/src/infrastructure`, { recursive: true });

// Domain entity — pure business logic, no external dependencies
Deno.writeTextFileSync(`${DEMO}/src/domain/user.ts`, `
export interface User {
  id: string;
  name: string;
  email: string;
}

export function createUser(name: string, email: string): User {
  return { id: Math.random().toString(36), name, email };
}
`.trimStart());

// Application use case — orchestrates domain objects
Deno.writeTextFileSync(`${DEMO}/src/application/register-user.ts`, `
import { createUser, User } from '../domain/user';

export function registerUser(name: string, email: string): User {
  const user = createUser(name, email);
  console.log('Registered:', user.name);
  return user;
}
`.trimStart());

// Infrastructure adapter — database repository
Deno.writeTextFileSync(`${DEMO}/src/infrastructure/user-repo.ts`, `
import { User } from '../domain/user';

const store: User[] = [];

export function saveUser(user: User): void {
  store.push(user);
}
`.trimStart());

Deno.writeTextFileSync(`${DEMO}/tsconfig.json`, JSON.stringify({
  compilerOptions: { target: "ES2020", module: "commonjs", strict: true, rootDir: "src", outDir: "dist" },
  include: ["src/**/*.ts"],
}, null, 2));

console.log("\nProject structure:");
await tree(DEMO);

### Step 1: Detect architecture

KindScript scans your directory structure and import graph to identify the pattern.

In [ ]:
await ksc("init", "--detect", DEMO);

### Step 2: Generate architecture definitions

`ksc infer` generates `architecture.ts` — the source of truth for your architectural rules. It uses `locate<T>()` to map Kind definitions to filesystem locations.

In [ ]:
await ksc("infer", "--write", DEMO);

In [ ]:
console.log("=== architecture.ts ===");
console.log(Deno.readTextFileSync(`${DEMO}/architecture.ts`));
console.log("=== kindscript.json ===");
console.log(Deno.readTextFileSync(`${DEMO}/kindscript.json`));

The generated `architecture.ts` has three sections:

1. **Kind definitions** — interfaces for the context and each layer
2. **Instance declaration** — `locate<CleanArchitectureContext>("src", { ... })` maps each member name to a directory. Member name `domain` → `src/domain/`, `application` → `src/application/`, etc.
3. **Contracts** — `defineContracts<T>()` declares the rules

The `locate<T>()` function is key: you only specify the root location (`"src"`), and member paths are derived automatically from the Kind definition tree.

### Step 3: Check contracts — should pass

In [ ]:
await ksc("check", DEMO);

### Step 4: Break a rule and see it caught

The domain layer should never depend on infrastructure. Let's add a forbidden import.

In [ ]:
Deno.writeTextFileSync(`${DEMO}/src/domain/user.ts`, `
import { saveUser } from '../infrastructure/user-repo';

export interface User {
  id: string;
  name: string;
  email: string;
}

export function createUser(name: string, email: string): User {
  const user = { id: Math.random().toString(36), name, email };
  saveUser(user); // BAD: domain reaching into infrastructure
  return user;
}
`.trimStart());

console.log("Violation introduced: domain/user.ts now imports from infrastructure");

In [ ]:
await ksc("check", DEMO);

KindScript caught it. The error tells you:
- **Where**: `src/domain/user.ts` line 1
- **What**: Forbidden dependency — domain is importing from infrastructure
- **Why**: The `noDependency(domain -> infrastructure)` contract
- **Error code**: `KS70001`

Exit code 1 means CI pipelines fail automatically.

### Step 5: Fix and verify

In [ ]:
Deno.writeTextFileSync(`${DEMO}/src/domain/user.ts`, `
export interface User {
  id: string;
  name: string;
  email: string;
}

export function createUser(name: string, email: string): User {
  return { id: Math.random().toString(36), name, email };
}
`.trimStart());

await ksc("check", DEMO);

Deno.removeSync(DEMO, { recursive: true });
console.log("Demo project cleaned up.");

---

## Part 2: New Project — Scaffold from Definitions

Starting fresh? Write `architecture.ts` first, then `ksc scaffold` creates the directory structure.

In [ ]:
const SCAFFOLD = Deno.makeTempDirSync({ prefix: "ksc-scaffold-" });

Deno.writeTextFileSync(`${SCAFFOLD}/tsconfig.json`, JSON.stringify({
  compilerOptions: { target: "ES2020", module: "commonjs", strict: true },
  include: ["src/**/*.ts", "architecture.ts"],
}, null, 2));

Deno.writeTextFileSync(`${SCAFFOLD}/kindscript.json`, JSON.stringify({
  definitions: ["architecture.ts"],
}, null, 2));

// Write architecture.ts with locate<T>() syntax
Deno.writeTextFileSync(`${SCAFFOLD}/architecture.ts`, `
interface Kind<N extends string = string> {
  readonly kind: N;
  readonly location: string;
}

type MemberMap<T extends Kind> = {
  [K in keyof T as K extends 'kind' | 'location' ? never : K]:
    T[K] extends Kind
      ? MemberMap<T[K]> | { path: string } & Partial<MemberMap<T[K]>> | Record<string, never>
      : never;
};
function locate<T extends Kind>(root: string, members: MemberMap<T>): MemberMap<T> {
  void root;
  return members;
}

interface ContractConfig {
  noDependency?: [string, string][];
  purity?: string[];
}
function defineContracts<_T = unknown>(config: ContractConfig): ContractConfig {
  return config;
}

export interface CleanContext extends Kind<"CleanContext"> {
  domain: DomainLayer;
  application: ApplicationLayer;
  infrastructure: InfrastructureLayer;
}

export interface DomainLayer extends Kind<"DomainLayer"> {}
export interface ApplicationLayer extends Kind<"ApplicationLayer"> {}
export interface InfrastructureLayer extends Kind<"InfrastructureLayer"> {}

export const app = locate<CleanContext>("src", {
  domain: {},
  application: {},
  infrastructure: {},
});

export const contracts = defineContracts<CleanContext>({
  noDependency: [
    ["domain", "infrastructure"],
    ["domain", "application"],
  ],
  purity: ["domain"],
});
`.trimStart());

console.log("Project defined (no src/ directory yet):");
await tree(SCAFFOLD);

In [ ]:
// Dry run: see what scaffold would create
await ksc("scaffold", SCAFFOLD);

In [ ]:
// Create the directories
await ksc("scaffold", "--write", SCAFFOLD);

console.log("\nProject structure after scaffold:");
await tree(SCAFFOLD);

In [ ]:
// Verify: ksc check passes on the freshly scaffolded project
await ksc("check", SCAFFOLD);

Deno.removeSync(SCAFFOLD, { recursive: true });

Scaffold is non-destructive — it skips existing directories and only creates what's missing. Safe to run repeatedly.

---

## Part 3: Advanced — Standalone Members, Nesting, and Path Overrides

For larger architectures, `locate<T>()` supports three advanced features.

### Standalone Members

Members can be defined as standalone typed constants, then composed into the parent. This improves readability for complex hierarchies.

```typescript
// Define members separately
const domain: DomainLayer = {
  entities: {},
  ports: {},
};

// Compose into the root instance
export const app = locate<AppContext>("src", {
  domain,           // standalone reference
  application: {},  // inline (fine for leaves)
  infrastructure: {},
});
```

The classifier resolves variable references — `domain` is looked up, its type annotation provides the Kind identity, and its initializer provides sub-member assignments.

### Nested Structure

Kind definitions can have nested sub-members. Path derivation is recursive:

```typescript
interface DomainLayer extends Kind<"DomainLayer"> {
  entities: EntitiesModule;
  ports: PortsModule;
}
```

With root `"src"`, the derived paths are:
- `domain` → `src/domain/`
- `domain.entities` → `src/domain/entities/`
- `domain.ports` → `src/domain/ports/`

### Path Overrides

By default, the member name is the directory name. If your filesystem uses a different convention, override per-member:

```typescript
const domain: DomainLayer = {
  valueObjects: { path: "value-objects" },  // src/domain/value-objects/ instead of src/domain/valueObjects/
  entities: {},
};
```

Override is per-member, per-instance. The Kind definition stays clean.

### Multi-Instance (Bounded Contexts)

The same Kind definition can be instantiated multiple times at different roots:

```typescript
export const ordering = locate<CleanContext>("src/ordering", {
  domain: {},
  application: {},
  infrastructure: {},
});

export const billing = locate<CleanContext>("src/billing", {
  domain: {},
  application: {},
  infrastructure: {},
});
```

`ordering.domain` resolves to `src/ordering/domain`. `billing.domain` resolves to `src/billing/domain`. Same structure, different positions.

---

## Summary

| Command | What it does |
|---------|-------------|
| `ksc init --detect` | Scans project, detects architectural pattern |
| `ksc infer --write` | Generates `architecture.ts` with kinds, locate instance, and contracts |
| `ksc scaffold --write` | Creates directory structure from definitions |
| `ksc check` | Validates all contracts — exit 0 if clean, exit 1 if violations |

**Typical workflows:**
- **Existing project**: `infer --write` → edit contracts → `check`
- **New project**: write `architecture.ts` → `scaffold --write` → implement → `check`
- **CI pipeline**: `check` on every PR

**Next:** See **02-contracts.ipynb** for all 5 contract types + existence checking